In [1]:
import matplotlib.pyplot as plt
import torch
import torchvision
import os


from torchvision import transforms

!pip install torchinfo

from torch import nn

In [2]:
!git clone https://github.com/shreyash-99/ERA1.git

Cloning into 'ERA1'...
remote: Enumerating objects: 368, done.
remote: Counting objects: 100% (368/368), done.
remote: Compressing objects: 100% (357/357), done.
remote: Total 368 (delta 9), reused 366 (delta 7), pack-reused 0
Receiving objects: 100% (368/368), 19.48 MiB | 23.86 MiB/s, done.
Resolving deltas: 100% (9/9), done.


In [3]:
cd ERA1/Session17/Assignment_solution/transformer

/content/ERA1/Session17/Assignment_solution/transformer


In [4]:
from transformer import GPT

In [5]:
from gpt_helper import (
    BATCH_SIZE,
    BLOCK_SIZE,
    DEVICE,
    DROPOUT,
    LEARNING_RATE,
    NUM_EMBED,
    NUM_HEAD,
    NUM_LAYER,
    MAX_ITER,
    EVAL_INTER,
    encode,
    decode,
    get_batch,
    save_model_to_chekpoint,
    estimate_loss,
)

In [6]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 29.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 52.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 68.2 MB/s eta 0:00:00


In [7]:
from transformers import AutoTokenizer

In [8]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [9]:
cd ../../SessionCodes

/content/ERA1/Session17/SessionCodes


In [10]:
# raw data
path_do_data = "GPT/data/english.txt"
data_raw = open(path_do_data, encoding="utf-8").read()
# we use pretrained BERT tokenizer for performance improvements
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
vocab_size = tokenizer.vocab_size
# data_raw = data_raw[4000000:] # short dataset

# train/val split
data = encode(text_seq=data_raw, tokenizer=tokenizer)
n = int(0.9 * len(data))  # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

Token indices sequence length is longer than the specified maximum sequence length for this model (37443 > 512). Running this sequence through the model will result in indexing errors


In [11]:
# train a new model
model = GPT(
    vocab_size=vocab_size,
    d_model=NUM_EMBED,
    block_size=BLOCK_SIZE,
    num_heads=NUM_HEAD,
    num_layers=NUM_LAYER,
    dropout=DROPOUT,
    device = DEVICE
)
# load model to GPU if available
m = model.to(DEVICE)
# print the number of parameters in the model
print(
    "Model with {:.2f}M parameters".format(sum(p.numel() for p in m.parameters()) / 1e6)
)

Model with 89.45M parameters


In [12]:
# optimizer takes the model's parameters and the learning rate as input,
# and updates the parameters during the training process in order to
# minimize the loss function.
optimizer = torch.optim.AdamW(m.parameters(), lr=LEARNING_RATE)
MAX_ITER = 500
for step in range(MAX_ITER):

    # every EVAL_INTER evaluate the loss on train and val sets
    if step % EVAL_INTER == 0 or step == MAX_ITER - 1:
        loss_train = estimate_loss(
            data=train_data, model=m, block_size=BLOCK_SIZE, batch_size=BATCH_SIZE
        )
        loss_val = estimate_loss(
            data=val_data, model=m, block_size=BLOCK_SIZE, batch_size=BATCH_SIZE
        )
        print("step {:10} | train loss {:6.4f} | val loss {:6.4f}".format(step, loss_train, loss_val))

    # sample a batch of data
    xb, yb = get_batch(data=train_data, block_size=BLOCK_SIZE, batch_size=BATCH_SIZE)
    logits, loss = m.forward(xb, yb)
    # zero_grad() method sets the gradients of all parameters in the optimizer to zero
    optimizer.zero_grad(set_to_none=True)
    # backward() method on the loss variable calculates the gradients
    # of the loss with respect to the model's parameters.
    loss.backward()
    # step() method on the optimizer updates the model's parameters
    # using the calculated gradients, in order to minimize the loss.
    optimizer.step()

step          0 | train loss 10.7437 | val loss 10.7569
step        499 | train loss 0.3488 | val loss 8.0341


In [13]:
save_model_to_chekpoint(model=m, path_to_checkpoint="checkpoint")

Successfully saved the model to checkpoint/checkpoint_epoch-0_08.10.2023_10:15:58.pt


In [14]:
# generate some output based on the context
context = torch.zeros((1, 1), dtype=torch.long, device=DEVICE)
print(
    decode(
        enc_sec=m.generate(idx=context, max_new_tokens=100, block_size=BLOCK_SIZE)[0],
        tokenizer=tokenizer,
    )
)

[PAD], we right now have covered in this case for : i should return your name this must have assignment 7 ( for resnet ) you are and more likely for all the best training accuracy is not fit not over a model, resnet'- error decreases from 6 of years. " memory storage " or a " from above you see " target should be passed on to the next layers, and what happens above topic? and output for example to the downstream tasks increases made. remember we must convince image
